In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import torch
import numpy as np 
import random as rn
import pandas as pd
import os
import sys
import datetime
import pkg_resources
import time
import gc
import re
import operator 
import sys
from sklearn import metrics
from sklearn import model_selection
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
from transformers import convert_bert_original_tf_checkpoint_to_pytorch
from transformers import ElectraConfig,BertConfig,BertTokenizer,BertForNextSentencePrediction, ElectraForSequenceClassification,BertForSequenceClassification,AdamW,BertModel,get_linear_schedule_with_warmup
from sklearn.metrics import roc_auc_score
from tqdm.notebook import *
import os
import logging
import tokenizers
import pickle
import shutil
import math
import random
import argparse
from sklearn.metrics import f1_score,accuracy_score,mean_absolute_error
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
import warnings
warnings.filterwarnings(action='once')


In [2]:
logger = logging.getLogger(__name__)

fh = logging.FileHandler("log/electra_large_v6.log", mode='w', encoding='utf-8')
LOG_FORMAT = '%(asctime)s [%(name)s] %(message)s'
logging.basicConfig(format = LOG_FORMAT, level = logging.INFO,datefmt = '%Y-%m-%d  %H:%M:%S %a' )
logger.addHandler(fh)

logger.info("...init...")

2020-12-03  04:22:21 Thu [__main__] ...init...


In [3]:
path="./data/"
os.listdir(path+"test/")

['test.query.tsv', 'test.reply.tsv']

In [4]:

train_query = pd.read_table(path+"train/train.query.tsv",names=['qid','query'])
train_reply = pd.read_table(path+"train/train.reply.tsv",names=["qid",'rid','reply','label'])

test_query = pd.read_table(path+"test/test.query.tsv",names=['qid','query'],encoding="gbk")
test_reply = pd.read_table(path+"test/test.reply.tsv",names=["qid",'rid','reply'],encoding="gbk")

In [5]:
train=pd.merge(train_query,train_reply,on="qid",how="left")
test=pd.merge(test_query,test_reply,on="qid",how="left")

In [6]:
train[train['reply'].isnull()]

,qid,query,rid,reply,label
2194,604,您好，请问这个房子周边有哪些学校,3,NaN,0


In [7]:
train=train[train['reply'].notnull()]

In [8]:
train.label.value_counts()

0    16197
1     5387
Name: label, dtype: int64

In [9]:
train['text']=train['query']+"_"+train['reply']
train['text_len']=train['text'].map(len)
train['text_len'].quantile(0.95)

41.0

In [10]:
test['text']=test['query']+"_"+test['reply']
test['text_len']=test['text'].map(len)
test['text_len'].quantile(0.999)

123.0

In [11]:
test['label']=0

In [12]:
train[['qid','rid','query','reply','label']].head()

,qid,rid,query,reply,label
0,0,0,采荷一小是分校吧,杭州市采荷第一小学钱江苑校区，杭州市钱江新城实验学校。,1
1,0,1,采荷一小是分校吧,是的,0
2,0,2,采荷一小是分校吧,这是5楼,0
3,1,0,毛坯吗？,因为公积金贷款贷的少,0
4,1,1,毛坯吗？,是呢,0


In [13]:
train.shape

(21584, 7)

In [14]:
# a=train.groupby("text",as_index=False)['label'].mean()
# b=a[(a.label!=0)&(a.label!=1) ]
# train = train[~train.text.isin(b.text.values)]
train.index=range(len(train))

In [15]:
train_group = train.groupby(['qid'])['reply'].apply(lambda x:" ".join(x)).reset_index()
train_group.columns=['qid','new_query']

train = pd.merge(train,train_group,on="qid",how='left')
train['text']=train['query']+"[SEP]"+train['new_query']+"_"+train['reply']
train['text_len']=train['text'].map(len)
train['text_len'].quantile(0.999)

305.4170000000013

In [16]:
test_group = test.groupby(['qid'])['reply'].apply(lambda x:" ".join(x)).reset_index()
test_group.columns=['qid','new_query']
test = pd.merge(test,test_group,on="qid",how='left')
test['text']=test['query']+"_"+test['new_query']+"_"+test['reply']
test['text_len']=test['text'].map(len)
test['text_len'].quantile(0.999)

325.0

In [17]:
#####for reproduction
def seed_everything(seed=1234):
    rn.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything()

In [18]:
# def read(path, batch_size, tokenizer,args,cache_size=1000000, shuffle=True,name=None):
#     def batchs(buf):
#         if shuffle: random.shuffle(buf)
#         batchs = int(math.ceil(float(len(buf)) / batch_size))
#         for i in range(batchs):
#             yield do_batch(buf[i * batch_size:(i + 1) * batch_size])

#     buf = []
#     with open(path, 'r', encoding='utf-8') as fin:
#         buf_size = 0
#         for line in fin:
#             line = line.rstrip('\r\n')
#             if len(line.split("\t"))<5:
#                 continue    
#             buf.append(do_line(tokenizer,line,args ))
#             buf_size += 1
#             if buf_size==1:
#                 print(buf[0])
#             if buf_size % 100000 == 0:
#                 if name is None:
#                     logger.info('Cache: #lines = %s' % buf_size)
#                 else:
#                     logger.info('%s Cache: #lines = %s' % (name, buf_size))
#             if buf_size == cache_size:
#                 logger.info('Cache: Done.')
#                 for batch in batchs(buf):
#                     yield batch
#                 buf = []
#                 buf_size = 0
#         if buf_size > 0:
#             for batch in batchs(buf):
#                 yield batch



# def do_line(tokenizer, text,args):
    
#     line = text.split('\t')
#     # print(line)
    
#     label = int(line[4])
#     query = line[2]
#     title = line[3]
#     token = tokenizer.encode(query,title)
    
#     one_token = token.ids
#     one_segment = token.type_ids
#     one_mask = token.attention_mask
    

#     assert len(one_token) == args['max_len']
#     assert len(one_segment) == args['max_len']
#     assert len(one_mask) == args['max_len']
#     return label, one_token,one_segment,one_mask

# def do_batch(batch_data):
#     batch_size = len(batch_data)

#     length = len(batch_data[0][1])

#     label = np.zeros([batch_size], dtype=np.int64)
    

#     token = np.zeros([batch_size, length], dtype=np.int64)
#     segment = np.zeros([batch_size, length], dtype=np.int64)
#     mask = np.zeros([batch_size, length], dtype=np.int64)
    

#     for i, ( l, t, s, m) in enumerate(batch_data):
#         label[i] = l
#         token[i, :] = t
#         segment[i, :] = s
#         mask[i, :] = m
       
       
    
#     label = torch.tensor(label, dtype=torch.long)
#     token = torch.tensor(token, dtype=torch.long)
#     segment = torch.tensor(segment, dtype=torch.long)
#     mask = torch.tensor(mask, dtype=torch.long)
   
#     return label, token, segment, mask

  

In [19]:
#######config
device=torch.device('cuda')
if not os.path.exists("roberta_large_pair"):
    os.mkdir("roberta_large_pair")

MAX_SEQUENCE_LENGTH = 220

WORK_DIR = "electra/"

# This is the Bert configuration file
bert_config = ElectraConfig.from_pretrained(WORK_DIR+'config.json',output_hidden_states=True,num_labels=2)

# bert_config = BertConfig.from_pretrained('hfl/chinese-roberta-wwm-ext-large',output_hidden_states=True,num_labels=2)

loading configuration file electra/config.json
Model config ElectraConfig {
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 1024,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "type_vocab_size": 2,
  "vocab_size": 21128
}



In [20]:
# Converting the lines to BERT format
# Thanks to https://www.kaggle.com/httpwwwfszyc/bert-in-keras-taming
def convert_lines(example,tokenizer,):
    all_tokens = []
    all_segments = []
    all_masks = []
#     max_len=[]
    for i,text in tqdm(example.iterrows()):
        
        tokens_a = tokenizer.encode(text['query']+"[SEP]"+text['new_query'],text['reply'])
        one_token = tokens_a.ids
        one_segment = tokens_a.type_ids
        one_mask = tokens_a.attention_mask
        
        
        all_tokens.append(one_token)
        all_segments.append(one_segment)
        all_masks.append(one_mask)
#         max_len.append(len(one_mask))
#     print(max(max_len))
    return np.array(all_tokens),np.array(all_segments),np.array(all_masks)

In [21]:
# !pip install tokenizers
import tokenizers
tokenizer = tokenizers.BertWordPieceTokenizer(
   WORK_DIR+"vocab.txt", 
    
    lowercase=True,
)
tokenizer.enable_truncation(MAX_SEQUENCE_LENGTH)
tokenizer.enable_padding(length=MAX_SEQUENCE_LENGTH)


tokens,segments,masks = convert_lines(train,tokenizer)
test_tokens,test_segments,test_masks = convert_lines(test,tokenizer)






/home/zhouqingsong/anaconda3/lib/python3.7/site-packages/tokenizers/implementations/bert_wordpiece.py:30: DeprecationWarning: Deprecated in 0.9.0: WordPiece.__init__ will not create from files anymore, try `WordPiece.from_file` instead
  tokenizer = Tokenizer(WordPiece(vocab, unk_token=str(unk_token)))


In [22]:
y = train['label'].values
features = [
    torch.tensor(tokens, dtype=torch.long),
    torch.tensor(segments, dtype=torch.long),
    torch.tensor(masks  , dtype=torch.long)
   
]
train_dataset = torch.utils.data.TensorDataset(*features, torch.tensor(y,dtype=torch.long))



test_features = [
    torch.tensor(test_tokens, dtype=torch.long),
    torch.tensor(test_segments, dtype=torch.long),
    torch.tensor(test_masks  , dtype=torch.long)
   
]
test_dataset = torch.utils.data.TensorDataset(*test_features)


In [23]:
def custom_loss(data, targets):
    ''' Define custom loss function for weighted BCE on 'target' column '''
    bce_loss = nn.CrossEntropyLoss()(data,targets)
    return bce_loss

def get_f1(pred,y_true):
    score = f1_score(y_true,pred.argmax(1))
    
    return score

In [24]:
def evalate(model,test_index,val_dataset,best,step,output_model_file,batch_size=128):
    model.eval()
    valid_preds = np.zeros((len(val_dataset),2))
    val_sampler = SequentialSampler(val_dataset)
    valid_loader = DataLoader(val_dataset, sampler=val_sampler, batch_size=batch_size)
    tk0 = tqdm_notebook(valid_loader)
    for i,(x_batch1,x_batch2,x_batch3, y_batch)   in enumerate(tk0):

        with torch.no_grad():
            pred = model(x_batch1.to(device),x_batch3.to(device),x_batch2.to(device))[0]#.logits
            valid_preds[i*batch_size:(i+1)*batch_size]=pred.detach().cpu().numpy()
    valid_preds=torch.softmax(torch.tensor(valid_preds),dim=1).numpy()
    f1 = get_f1(valid_preds, y[test_index])#accuracy_score(y[test_index],valid_preds.argmax(1))
    logger.info("STEP {} | F1 {:4f} ".format(step,f1))
    if f1>best:
        best=f1
        model_to_save = model.module if hasattr(model, 'module') else model
        torch.save(model_to_save.state_dict(), output_model_file)
        logger.info("save best model.....bset score is {}  ".format(best))

    logger.info("***"*30)
    return best
    

In [25]:
def predict(i,train_index, test_index,batch_size=128):
    #seed_everything(i)
    tr_dataset = torch.utils.data.Subset(train_dataset, train_index)
    val_dataset = torch.utils.data.Subset(train_dataset, test_index)
    output_model_file = WORK_DIR+"electra_large_pytorch_{}.bin".format(i)
    model = ElectraForSequenceClassification.from_pretrained(output_model_file,config=bert_config)
    model.to(device)
    for param in model.parameters():
        param.requires_grad=False
        
    
    ##val
    valid_preds = np.zeros((len(val_dataset),2))
    val_sampler = SequentialSampler(val_dataset)
    valid_loader = DataLoader(val_dataset, sampler=val_sampler, batch_size=batch_size)
    tk0 = tqdm_notebook(valid_loader)
    for i,(x_batch1,x_batch2,x_batch3, y_batch)   in enumerate(tk0):
        model.eval()
        with torch.no_grad():
            pred = model(x_batch1.to(device),x_batch3.to(device),x_batch2.to(device))[0]#.logits
            valid_preds[i*batch_size:(i+1)*batch_size]=pred.detach().cpu().numpy()
    valid_preds=torch.softmax(torch.tensor(valid_preds),dim=1).numpy()
    f1= get_f1(valid_preds, y[test_index])#accuracy_score(y[test_index],valid_preds.argmax(1))
    print(" F1 {:4f} ".format(f1))
    #test
    test_preds = np.zeros((len(test_dataset),2))
    test_sampler = SequentialSampler(test_dataset)
    test_loader = DataLoader(test_dataset, sampler=test_sampler, batch_size=batch_size)
    tk0 = tqdm_notebook(test_loader)
    for i,(x_batch1,x_batch2,x_batch3,)   in enumerate(tk0):
        model.eval()
        with torch.no_grad():
            pred =model(x_batch1.to(device),x_batch3.to(device),x_batch2.to(device))[0]#.logits
            test_preds[i*batch_size:(i+1)*batch_size]=pred.detach().cpu().numpy()

    test_preds=torch.softmax(torch.tensor(test_preds),dim=1).numpy()
        
    return valid_preds,test_preds,f1
    


In [26]:
class FGM():
    def __init__(self, model):
        self.model = model
        self.backup = {}

    def attack(self, epsilon=1., emb_name='word_embeddings'):
        # emb_name这个参数要换成你模型中embedding的参数名
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name:
                self.backup[name] = param.data.clone()
                norm = torch.norm(param.grad)
                if norm != 0:
                    r_at = epsilon * param.grad / norm
                    param.data.add_(r_at)

    def restore(self, emb_name='word_embeddings'):
        # emb_name这个参数要换成你模型中embedding的参数名
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name: 
                assert name in self.backup
                param.data = self.backup[name]
        self.backup = {}

In [27]:

def train_model(i,train_index, test_index,train_dataset=train_dataset):
    seed_everything(i)
    fold = i
    output_model_file = WORK_DIR+"electra_large_pytorch_{}.bin".format(i)
    lr=1e-5
    batch_size = 8
    max_grad_norm =1.0
    gradient_accumulation_steps=1
    EPOCHS = 5
    multi_gpu=False
    fp16=False
    
    tr_dataset =torch.utils.data.Subset(train_dataset, train_index)
    val_dataset = torch.utils.data.Subset(train_dataset, test_index)
    
    train_sampler = RandomSampler(tr_dataset)
    train_loader = DataLoader(tr_dataset, sampler=train_sampler, batch_size=batch_size)

    model = ElectraForSequenceClassification.from_pretrained(WORK_DIR+"pytorch_model.bin",config=bert_config)
    model.zero_grad()
    
    model = model.to(device)
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias','LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
    
    t_total = len(train_loader) // gradient_accumulation_steps * EPOCHS
    optimizer=AdamW(optimizer_grouped_parameters,lr=lr)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=int(0.0*t_total), num_training_steps=t_total)
    if fp16:
        model, optimizer = amp.initialize(model, optimizer, opt_level="O1",verbosity=0)
    if multi_gpu:
        model = torch.nn.DataParallel(model)
    best = 0
    logger.info("***** begin train *****")
    tq = tqdm_notebook(range(EPOCHS))
    step=0
    for epoch in tq:
        avg_loss = 0.
        avg_accuracy = 0.
        
        lr = optimizer.state_dict()['param_groups'][0]['lr']
        start = time.strftime("%H:%M:%S")
        logger.info("Starting epoch: %d | phase: train | ⏰: %s | Learning rate: %f" % (epoch+1, start, lr))

        lossf=None
        tk0 = tqdm_notebook(enumerate(train_loader),total=len(train_loader),leave=False)
#         optimizer.zero_grad()  
        fgm = FGM(model)
        for i,(x_batch1,x_batch2,x_batch3, y_batch) in tk0:
            step+=1
            model.train()
            y_pred = model(x_batch1.to(device),x_batch3.to(device),x_batch2.to(device))[0]#.logits
            loss =  custom_loss(y_pred,y_batch.to(device))
            if multi_gpu:
                loss = loss.mean()
            if gradient_accumulation_steps > 1:
                loss = loss / gradient_accumulation_steps
            if fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
                torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), max_grad_norm)
            else:
                loss.backward()
                fgm.attack() # 在embedding上添加对抗扰动
                y_pred = model(x_batch1.to(device),x_batch3.to(device),x_batch2.to(device))[0]
                loss_adv =  custom_loss(y_pred,y_batch.to(device))
                loss_adv.backward() # 反向传播，并在正常的grad基础上，累加对抗训练的梯度
                fgm.restore() # 恢复embedding参数

                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            if (i+1) % gradient_accumulation_steps == 0:             # Wait for several backward steps
                optimizer.step()
                scheduler.step()
                model.zero_grad()
            if lossf:
                lossf = 0.98*lossf+0.02*loss.item()
            else:
                lossf = loss.item()
            
           
            tk0.set_postfix(loss = lossf,)
            avg_loss += loss.item() / len(train_loader)
           
            tq.set_postfix(avg_loss=avg_loss)
            
            # if step%5000==0:
        best=evalate(model,test_index,val_dataset,best,epoch+1,output_model_file)
    
    logger.info("======predict=========")
    valid_preds,test_preds,best=predict(fold,train_index, test_index)
    
    return valid_preds,test_preds,best

In [28]:
from sklearn.model_selection import StratifiedKFold,KFold,GroupKFold
kfold = GroupKFold(n_splits=5)
oof_preds=np.zeros((len(train),2))
test_preds=0
scores=[]
for i, (train_index, test_index) in enumerate(kfold.split(np.zeros(len(train)),y,train['qid'])):
    # if i<=3:continue
    torch.cuda.empty_cache()
    gc.collect()
    logger.info("FOLD  {}|{}".format(i+1,5))
    logger.info("###"*30)
    preds,test_temp,score=train_model(i,train_index, test_index)
    # preds,test_temp,score = predict(i,train_index, test_index)
    oof_preds[test_index]=preds
    test_preds+=test_temp/5
    scores.append(score)
    



    
    

2020-12-03  04:23:54 Thu [__main__] FOLD  1|5
2020-12-03  04:23:54 Thu [__main__] ##########################################################################################
loading weights file electra/pytorch_model.bin
Some weights of the model checkpoint at electra/pytorch_model.bin were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a 

2020-12-03  04:24:15 Thu [__main__] Starting epoch: 1 | phase: train | ⏰: 04:24:15 | Learning rate: 0.000010


2020-12-03  04:51:14 Thu [__main__] STEP 1 | F1 0.770115 


2020-12-03  04:51:24 Thu [__main__] save best model.....bset score is 0.7701149425287356  
2020-12-03  04:51:25 Thu [__main__] ******************************************************************************************
2020-12-03  04:51:25 Thu [__main__] Starting epoch: 2 | phase: train | ⏰: 04:51:25 | Learning rate: 0.000008


2020-12-03  05:18:19 Thu [__main__] STEP 2 | F1 0.781621 


...

2020-12-03  16:03:59 Thu [__main__] STEP 4 | F1 0.795956 
2020-12-03  16:03:59 Thu [__main__] ******************************************************************************************
2020-12-03  16:03:59 Thu [__main__] Starting epoch: 5 | phase: train | ⏰: 16:03:59 | Learning rate: 0.000002


2020-12-03  16:31:12 Thu [__main__] STEP 5 | F1 0.800000 


2020-12-03  16:31:20 Thu [__main__] save best model.....bset score is 0.8  
2020-12-03  16:31:20 Thu [__main__] ******************************************************************************************
2020-12-03  16:31:20 Thu [__main__] ======predict=========
loading weights file electra/electra_large_pytorch_4.bin


All model checkpoint weights were used when initializing ElectraForSequenceClassification.

All the weights of ElectraForSequenceClassification were initialized from the model checkpoint at electra/electra_large_pytorch_4.bin.
If your task is similar to the task the model of the checkpoint was trained on, you can already use ElectraForSequenceClassification for predictions without further training.



 F1 0.800000 


In [39]:
print("mean score:",np.mean(scores))

mean score: 0.8077070629184269


In [40]:
scores

[0.7977207977207977,
 0.8125000000000001,
 0.8153214774281806,
 0.8129930394431555,
 0.8]

In [41]:
def search_f1(y_true, y_pred):
    best = 0
    best_t = 0
    for i in range(30,60):
        tres = i / 100
        y_pred_bin =  (y_pred > tres).astype(int)
        score = f1_score(y_true, y_pred_bin)
        if score > best:
            best = score
            best_t = tres
    print('best', best)
    print('thres', best_t)
    return best, best_t

In [42]:
best_score, best_t = search_f1(train['label'].values,oof_preds[:,1])

best 0.8090681036885626
thres 0.35


In [43]:
oof = pd.DataFrame(oof_preds,columns=['pred0',"pred1"])
oof['label']=train['label'].values
oof['qid']=train['qid'].values
oof['rid']=train["rid"].values
if not os.path.exists("oof"):
    os.mkdir("oof")
oof.to_csv("./oof/electra_large_oof_v6.csv",index=False)

In [44]:
sub = test[['qid','rid']]
sub['pred'] = test_preds[:,1]
if not os.path.exists("submit"):
    os.mkdir("submit")

sub.to_csv("./submit/submit_electra_large_pred_v6.csv",index=False)
sub['label'] = sub['pred'].map(lambda x:1 if x>=best_t   else 0)
sub[['qid','rid','label']].to_csv("./submit/submit_electra_large_v6_{}.csv".format(best_score),sep="\t",index=None,header=None)

    

/home/zhouqingsong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [45]:
sub

,qid,rid,pred,label
0,0,0,0.000066,0
1,0,1,0.000050,0
2,0,2,0.000039,0
3,0,3,0.000051,0
4,0,4,0.000032,0
5,1,0,0.000033,0
6,1,1,0.999687,1
7,1,2,0.000364,0
8,2,0,0.000057,0
9,2,1,0.000336,0


In [46]:
sub['label'].value_counts()

0    40674
1    13083
Name: label, dtype: int64

In [47]:
train['label'].value_counts()/len(train)

0    0.750417
1    0.249583
Name: label, dtype: float64

In [48]:
!nvidia-smi

Thu Dec  3 21:47:07 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.118.02   Driver Version: 440.118.02   CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000000:3F:00.0 Off |                    0 |
| N/A   35C    P0    37W / 250W |  15737MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  Off  | 00000000:40:00.0 Off |                    0 |
| N/A   61C    P0   199W / 250W |  15539MiB / 16160MiB |     37%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T